In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv
import os
import logging
import pickle

In [15]:
logging.basicConfig(level=logging.INFO)

def load_all_dataframes():
    try:
        grammy_df = pd.read_pickle('./grammy_ready_df.pkl')
        spotify_df = pd.read_pickle('./spotify_ready_df.pkl')
        
        logging.info(f"Grammy dataframe: \n{grammy_df.head(5)}")
        logging.info(f"Spotify dataframe: \n{spotify_df.head(5)}")
        
        logging.info("Dataframes loaded successfully")
        
        return grammy_df, spotify_df
        
        
    except Exception as e:
        logging.error(f"Error loading dataframes: {e}")

load_all_dataframes()

INFO:root:Grammy dataframe: 
   year                              title              published_at  \
0  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
1  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
2  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
3  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   
4  2019  62nd Annual GRAMMY Awards  (2019) 2020-05-19 12:10:28+00:00   

                 updated_at            category     nominee         artist  \
0 2020-05-19 12:10:28+00:00  Record Of The Year     Bad Guy  Billie Eilish   
1 2020-05-19 12:10:28+00:00  Record Of The Year     Hey, Ma       Bon Iver   
2 2020-05-19 12:10:28+00:00  Record Of The Year     7 rings  Ariana Grande   
3 2020-05-19 12:10:28+00:00  Record Of The Year  Hard Place         H.E.R.   
4 2020-05-19 12:10:28+00:00  Record Of The Year        Talk         Khalid   

                                             workers  winner  
0  Fin